<a href="https://colab.research.google.com/github/vsattamini/silvertone/blob/luiz/model_fourrier_mel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tensorflow.keras import layers, Sequential, Input, Model
from  tensorflow import expand_dims
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils, to_categorical
from sklearn.preprocessing import MinMaxScaler
import torch
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_pickle("/content/drive/MyDrive/df/database_final_v4_6emo.pkl")

In [3]:
y = df["emotion"]
X = df[["mel_spec", "fourrier","intensity", "repetition", "actor_gender"]]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1405, stratify=y)

In [5]:
X_train_mel = np.array(X_train["mel_spec"].values.tolist()).astype('float32')
X_train_fourrier = np.array(X_train["fourrier"].values.tolist()).astype('float32')

In [6]:
X_test_mel = np.array(X_test["mel_spec"].values.tolist()).astype('float32')
X_test_fourrier = np.array(X_test["fourrier"].values.tolist()).astype('float32')

In [7]:
X_train_mel = expand_dims(X_train_mel, -1)
X_train_fourrier = expand_dims(X_train_fourrier, -1)

In [8]:
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

In [9]:
input_mel = Input(shape=(X_train_mel.shape[1:]))
input_fourrier = Input(shape=(X_train_fourrier.shape[1:]))

In [10]:
mel = layers.Conv2D(64, (4,4), activation="relu", padding="same")(input_mel)
mel = layers.MaxPool2D((2,2))(mel)
mel = layers.Conv2D(128, (3,3), activation="relu", padding="same")(mel)
mel = layers.Conv2D(128, (3,3), activation="relu", padding="same")(mel)
mel = layers.MaxPool2D((2,2))(mel)
mel = layers.Conv2D(128, (3,3), activation="relu", padding="same")(mel)
mel = layers.Conv2D(128, (3,3), activation="relu", padding="same")(mel)
mel = layers.Flatten()(mel)
mel = Model(inputs=input_mel, outputs=mel)

In [11]:
fourrier = layers.Conv2D(64, (4,4), activation="relu", padding="same")(input_fourrier)
fourrier = layers.MaxPool2D((2,2))(fourrier)
fourrier = layers.Conv2D(128, (3,3), activation="relu", padding="same")(fourrier)
fourrier = layers.Conv2D(128, (3,3), activation="relu", padding="same")(fourrier)
fourrier = layers.MaxPool2D((2,2))(fourrier)
fourrier = layers.Conv2D(128, (3,3), activation="relu", padding="same")(fourrier)
fourrier = layers.Conv2D(128, (3,3), activation="relu", padding="same")(fourrier)
fourrier = layers.Flatten()(fourrier)
fourrier = Model(inputs=input_fourrier, outputs=fourrier)

In [12]:
combined = layers.Concatenate()([mel.output, fourrier.output])

In [13]:
final = layers.Dense(10, activation="relu")(combined)
final = layers.Dense(7, activation="softmax")(final)

In [14]:
model = Model(inputs=[mel.input, fourrier.input], outputs=final)

In [15]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
es = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(x=[X_train_mel, X_train_fourrier], y = y_train, validation_split=0.3, epochs=500, batch_size=32, callbacks=[es])

Epoch 1/500
206/206 [==============================] - 1422s 7s/step - loss: 1.8288 - accuracy: 0.2513 - val_loss: 1.5743 - val_accuracy: 0.3352
Epoch 2/500
206/206 [==============================] - 1422s 7s/step - loss: 1.4750 - accuracy: 0.3968 - val_loss: 1.4134 - val_accuracy: 0.4701
Epoch 3/500
206/206 [==============================] - 1391s 7s/step - loss: 1.3388 - accuracy: 0.4873 - val_loss: 1.3576 - val_accuracy: 0.4945
Epoch 4/500
206/206 [==============================] - 1380s 7s/step - loss: 1.2134 - accuracy: 0.5324 - val_loss: 1.3391 - val_accuracy: 0.4733
Epoch 5/500
206/206 [==============================] - 1388s 7s/step - loss: 1.1759 - accuracy: 0.5315 - val_loss: 1.3638 - val_accuracy: 0.5005
Epoch 6/500
206/206 [==============================] - 1399s 7s/step - loss: 1.0424 - accuracy: 0.5855 - val_loss: 1.3471 - val_accuracy: 0.5250
Epoch 7/500
206/206 [==============================] - 1416s 7s/step - loss: 0.8977 - accuracy: 0.6483 - val_loss: 1.5282 - val_ac